In [ ]:
import numpy as np

In [ ]:
def preparedata_time_window(input_data,window):
    out_data = []
    # rearrange the input in form of output based on 3 observations in past
    for i in range(len(input_data)):
        if (i+window+1) <= len(input_data):
            out_data.append((input_data[i:i+window],input_data[i+window:i+(window+1)]))
            
    return out_data

In [ ]:
import pandas as pd

In [ ]:
import torch
from torch import nn     
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# df = pd.read_csv('https://raw.githubusercontent.com/riteshsv/simplelstm/main/tsdata.csv')
# df.columns=['sales']
# df.head()

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/riteshsv/simplelstm/main/ULTRACEMCO.NS.csv')
# df.columns=['sales']
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-11-16,8120.0,8139.000000,7856.299805,7882.399902,7837.409180,277983
1,2021-11-17,7845.0,7948.950195,7801.000000,7857.700195,7812.850586,183623
2,2021-11-18,7899.0,7930.000000,7715.049805,7767.700195,7723.364258,196805
3,2021-11-22,7799.0,7808.600098,7550.600098,7653.799805,7610.113770,383629
4,2021-11-23,7620.0,7714.700195,7540.700195,7657.950195,7614.240723,475377


In [ ]:
df['Date'].min()

'2021-11-16'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = df['Close'].values.astype(float)

In [ ]:
data[-14:]

array([6436.799805, 6445.850098, 6714.950195, 6820.899902, 6779.899902,
       6728.25    , 6893.049805, 6973.5     , 6903.5     , 6801.299805,
       6813.200195, 6854.399902, 6964.549805, 6938.350098])

In [ ]:
# X = np.array(X)
# y = np.array(y)

In [ ]:
# scaler = MinMaxScaler(feature_range=(-1,1))
# scaled_input = scaler.fit_transform( np.array(input_data).reshape(-1,1))

In [ ]:
# train_data = preparedata_time_window(torch.tensor(scaled_input,dtype=torch.float32).view(-1),3)

In [ ]:
class LSTMSimple(nn.Module):
    def __init__(self, input_dim,hidden_dim,out_size,num_layers=1,batch_size=1):
        super(LSTMSimple,self).__init__()
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim,out_size)
        self.hidden = (torch.zeros(1*num_layers,batch_size,hidden_dim),torch.zeros(1*num_layers,batch_size,hidden_dim))
    def forward(self,x):
        out,self.hidden = self.lstm(x.view(len(x),self.batch_size,-1),self.hidden)
        preds = self.out(out.view(len(x),-1))
        return preds[0]
        

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
model = LSTMSimple(1,50,1)
model.to(device)

LSTMSimple(
  (lstm): LSTM(1, 50)
  (out): Linear(in_features=50, out_features=1, bias=True)
)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(),lr=1e-10)

In [ ]:
def train(train_data):
    for epoch in range(100):
        for x,t in train_data:
            model.zero_grad()
            model.hidden = (torch.zeros(1,1,50,device=device),torch.zeros(1,1,50,device=device))

            preds = model(x)
            loss = loss_fn(preds,t)
            loss.backward()
            optimizer.step()
        if epoch%25 == 0:
            print(f"epoch: {epoch} loss:{loss.item()}")
        

        

In [ ]:
scaler2 = MinMaxScaler(feature_range=(-1,1))
scaled_input2 = scaler2.fit_transform(data.reshape(-1,1))
train_data2 = preparedata_time_window(torch.tensor(scaled_input2,dtype=torch.float32,device=device).view(-1),14)

In [ ]:
train(train_data2)

epoch: 0 loss:0.09602939337491989
epoch: 25 loss:0.09602939337491989
epoch: 50 loss:0.09602939337491989
epoch: 75 loss:0.09602939337491989


In [ ]:
with torch.no_grad():
  input_ten = torch.tensor(scaled_input2[-14:],dtype=torch.float32,device=device).view(-1)
  model.hidden = (torch.zeros(1,1,50,device=device),torch.zeros(1,1,50,device=device))
  preds = model(input_ten)
  print(preds)
  actual_pred = scaler2.inverse_transform(np.array(preds.view(-1).cpu()).reshape(-1,1))

print(actual_pred)


tensor([-0.0080], device='cuda:0')
[[6519.0356]]


In [1]:
from matplotlib import pyplot as plt

In [ ]:
plt.title('Stock Price')
plt.ylabel('Price')
plt.grid(True)
plt.autoscale(axis='x', tight=True)
plt.plot(flight_data['passengers'])
plt.plot(x,actual_predictions)
plt.show()

In [2]:
# !git clone https://github.com/riteshsv/simplelstm.git

Cloning into 'simplelstm'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


# New Section